# PSI3471 - Entrega 01 
### Fernando Zolubas Preto - NUSP: 10694192
### Vinícius Melo de Souza - NUSP: 10772272
# Regressão Linear 

# Tarefas

- 0) Intuir combinação de variaveis ótima

- 1) Criar Função que dada uma tabela de dados retorna a regressão para preço

- 2) Dummyzar variaveis qualitativas

- 3) Criar Variaveis Novas

- 4) Chamar Função para todas as combinações de variaveis, salvar predição da base testes e escolher melhor.

- 5) Comparar resultado da intuição versus melhor resultado numerico obtido

Neste exercício queremos prever o preço de carros do _"Used Cars Dataset"_ do site `Craiglist.org`.

No caso, os dados já foram limpos e filtrados para facilitar o entendimento e o andamento do exercício, para conseguirmos dar foco na utilização da técnica dos mínimos quadrados. Os dados limpos estão em um arquivo CSV que podemos puxar do site da disciplina. Este arquivo será importado através da biblioteca "Panda" do python que foi instalada no computador previamente.

In [56]:
import pandas as pd
import numpy as np
cars_data = pd.read_csv("vehicles_cleaned_train.csv")

Com esses dados, queremos criar um _"DataFrame"_ que estaremos chamando de __cars_data__, que irá conter esses dados do arquivo CSV. Neste _DataFrame_ estaremos gerando os arrays NumPy para calcular os parâmetros do modelo de regressão linear.

O banco de dados limpo possui os seguintes dados(exemplo):

In [57]:
cars_data.head()

,price,year,condition,cylinders,fuel,odometer,transmission,size,type
0,10400,2011,excellent,4 cylinders,gas,81300,automatic,mid-size,sedan
1,6900,2007,excellent,6 cylinders,gas,79000,automatic,full-size,sedan
2,18900,2018,like new,4 cylinders,gas,5000,automatic,full-size,sedan
3,6000,2010,excellent,4 cylinders,gas,97600,automatic,mid-size,hatchback
4,19995,2013,good,6 cylinders,gas,95782,automatic,mid-size,sedan


Temos essas 9 colunas que se referem à:

- __Price__: O preço do carro. É o dado que desejamos obter com o modelo.
- __Year__: Ano do carro.
- __Condition__: Variável categórica que indica a condição do carro. Pode ter os valores _good_, _fair_, _excellent_, _like new_, _salvage_, ou _new_.
- __Cylinders__: Variável categórica que indica o número de cilindros do motor. Pode ter os valores _4 cylinders_ ou _6 cylinders_.
- __Fuel__: Variável categórica que indica o combustível do carro. Pode ter os valores _gas_ ou _diesel_.
- __Odometer__: Valor registrado no odômetro, em milhas.
- __Transmission__: Variável categórica que indica o tipo de transmissão. Pode ter os valores _automatic_ ou _manual_.
- __Size__: Variável categórica que indica o tamanho do carro. Pode ter os valores _compact_, _mid-size_, _sub-compact_ ou _full-size_.
- __Type__:	Variável categórica que indica o tipo do carro. Pode ter os valores _sedan_, _coupe_, _wagon_, ou _hatchback_.


Para usar os dados categóricos de forma numérica iremos estar utilizando variáveis _dummy_.

E por fim, para obter o vetor "wo" vamos seguir os seguintes passos.

1. Selecionar o conjunto de variáveis originais que vocễ vai utilizar no modelo.
2. Substituir cada variável categórica de sua seleção por um conjunto de variáveis dummy, conforme descrito anteriormente.
3. Transformar as variáveis originais de sua seleção e / ou incluir combinações, caso julgue necessário;
4. A partir de sua seleção de dados, obter a matriz X e o vetor d, que podem ser representados como arrays do NumPy.
5. Usando a matriz X e o vetor d, calcular o vetor wo e o erro quadrático médio conforme mostrado na aula.

# Nossa solução

1. Função de regressão linear

In [58]:
def RL_reta(data):
    """
    Função que implementa a regressão linear multipla para prever o preço de carros.

    dados é um dataframe com as colunas:
    - price: O preço do carro.
    - year: O ano do carro.
    - condition: Variável categórica que indica a condição do carro. #CATEGORICA
    - cylinders: Variável categórica que indica o número de cilindros do motor. #CATEGORICA
    - fuel: Variável categórica que indica o combustível do carro. #CATEGORICA
    - odometer: Valor registrado no odômetro, em milhas.
    - transmission: Variável categórica que indica o tipo de transmissão. #CATEGORICA
    - size: Variável categórica que indica o tamanho do carro. #CATEGORICA
    - type: Variável categórica que indica o tipo do carro. #CATEGORICA

    """

    # Extrai as colunas relevantes dos dados de entrada
    fabrication = data['year'].values.reshape(-1, 1)
    odometer = data['odometer'].values.reshape(-1, 1)
    d = data['price'].values.reshape(-1, 1)

    # Cria as variáveis dummy para as variáveis categóricas
    condition_dummy = pd.get_dummies(data['condition'])
    cylinders_dummy = pd.get_dummies(data['cylinders'])
    fuel_dummy = pd.get_dummies(data['fuel'])
    transmission_dummy = pd.get_dummies(data['transmission'])
    size_dummy = pd.get_dummies(data['size'])
    type_dummy = pd.get_dummies(data['type'])

    # Nossas novas variáveis
    # 1. Idade do carro ao inves de ano de fabricação
    current_year = 2023
    idade = current_year - fabrication
    # 2. Milhagem por anos ao invés de só milhagem
    milhagem_anos = odometer/idade

    # Concatena as variáveis dummy com os dados de entrada originais
    x = np.hstack([fabrication, odometer, condition_dummy, cylinders_dummy, fuel_dummy, transmission_dummy, size_dummy, type_dummy])

    # N representa o número de dados para o cálculo da RL
    N = d.shape[0]

    # X é a matriz dos dados
    X = np.hstack([np.ones((N, 1)), x])

    # R é o produto das matrizes X.T e X
    R = X.T @ X

    # o vetor p é dado pelo produto entre a matriz X.T e o vetor d
    p = X.T @ d

    # Calcule a solução wo e o erro e
    wo = np.linalg.solve(R, p)
    e = d - X @ wo

    return wo, e

2. Com isso agora devemos ter os parâmetros da regressão:

In [59]:
wo, e = RL_reta(cars_data)

3. Agora precisamos rodar com os dados reais para testar o nosso modelo

Nossos dados reais estão no csv "vehicles_cleaned_train.csv"

In [60]:
cars_data_teste = pd.read_csv("vehicles_cleaned_train.csv")
cars_data_teste.head()

,price,year,condition,cylinders,fuel,odometer,transmission,size,type
0,10400,2011,excellent,4 cylinders,gas,81300,automatic,mid-size,sedan
1,6900,2007,excellent,6 cylinders,gas,79000,automatic,full-size,sedan
2,18900,2018,like new,4 cylinders,gas,5000,automatic,full-size,sedan
3,6000,2010,excellent,4 cylinders,gas,97600,automatic,mid-size,hatchback
4,19995,2013,good,6 cylinders,gas,95782,automatic,mid-size,sedan


Novamente precisamos criar as variaveis dummy que usamos anteriormente e preparar os vetores X e d com os dados de teste

In [61]:
# criar variáveis dummy para as variáveis categóricas para os dados
condition_dummy = pd.get_dummies(cars_data_teste['condition'])
cylinders_dummy = pd.get_dummies(cars_data_teste['cylinders'])
fuel_dummy = pd.get_dummies(cars_data_teste['fuel'])
transmission_dummy = pd.get_dummies(cars_data_teste['transmission'])
size_dummy = pd.get_dummies(cars_data_teste['size'])
type_dummy = pd.get_dummies(cars_data_teste['type'])

In [62]:
# Dados quantitativos
fabrication_teste = cars_data_teste['year'].values.reshape(-1, 1)
odometer_teste = cars_data_teste['odometer'].values.reshape(-1, 1)
# Nossas novas variáveis
    # 1. Idade do carro ao inves de ano de fabricação
current_year = 2023
idade_teste = current_year - fabrication_teste
    # 2. Milhagem por anos ao invés de só milhagem
milhagem_anos_teste = odometer_teste/idade_teste

xTeste = np.hstack([fabrication_teste, odometer_teste, condition_dummy, cylinders_dummy, fuel_dummy, transmission_dummy, size_dummy, type_dummy])

# d é o vetor com os preços reais
dTeste = cars_data_teste['price'].values.reshape(-1, 1)

# N representa o número de dados para o cálculo da RL
N = dTeste.shape[0]

# X é a matriz dos dados
XTeste = np.hstack([np.ones((N, 1)), xTeste])

E agora podemos fazer a previsão de preço para cada carro:

In [63]:
# previsoes e erros
previsto = []
erro = []

# para cada linha do conjunto de teste
for i in range(len(cars_data_teste)):
    # extrai as features da linha i
    xTeste_i = XTeste[i, :]
    # calcula o valor predito pelo modelo
    y_pred_i = xTeste_i @ wo
    # calcula o erro em relação ao valor real
    e_i = dTeste[i] - y_pred_i
    # salva nos vetores
    # imprime o valor predito, o valor real e o erro
    print(f"Valor predito: {y_pred_i}, Valor real: {dTeste[i]}, Erro: {e_i}")

Valor predito: [8359.85042598], Valor real: [10400], Erro: [2040.14957402]
Valor predito: [8124.08066529], Valor real: [6900], Erro: [-1224.08066529]
Valor predito: [17891.02183274], Valor real: [18900], Erro: [1008.97816726]
Valor predito: [6890.73753292], Valor real: [6000], Erro: [-890.73753292]
Valor predito: [10795.19054783], Valor real: [19995], Erro: [9199.80945217]
Valor predito: [455.84531232], Valor real: [3400], Erro: [2944.15468768]
Valor predito: [12684.00369914], Valor real: [13900], Erro: [1215.99630086]
Valor predito: [7205.04265067], Valor real: [7500], Erro: [294.95734933]
Valor predito: [11150.6243575], Valor real: [9400], Erro: [-1750.6243575]
Valor predito: [17905.32366938], Valor real: [12500], Erro: [-5405.32366938]
Valor predito: [16378.16721839], Valor real: [18500], Erro: [2121.83278161]
Valor predito: [12974.86053554], Valor real: [11200], Erro: [-1774.86053554]
Valor predito: [13958.30369896], Valor real: [24500], Erro: [10541.69630104]
Valor predito: [15044

E por fim o erro quadrático médio:

In [64]:
mse = np.mean((dTeste - previsto)**2)
print(f"Erro quadrático médio: {mse}")

Erro quadrático médio: nan


c:\Users\vmelo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\vmelo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [65]:
#Celula do zolubas


In [66]:
def regressaoLinear(X,d):
    """ Função que implementa a regressão linear multivariada
    X é a matriz que contém os dados de treinamento
    d é o vetor que contem o sinal desejado ou rótulo """

    R = X.T @ X
    p = X.T @ d

    """ Sistema Linear Ax = b 
    A = R , b = p 
    wo = Optimal Weights: Solucao do metodo minimos quadrados"""

    wo = np.linalg.solve(R,p)
    """ Vetor de Erros do metodo dos minimos quadrados """
    e = d - X @ wo

    return wo, e

In [67]:
for i in combinacoes_index
    wo, e = regressaoLinear(X(:,i),d)

SyntaxError: expected ':' (1210209190.py, line 1)